In [43]:
import json
import os
import glob
import re
import pandas as pd
from os import makedirs
from os.path import join, exists
from pathlib import Path
import numpy as np

In [89]:
states = ['US', 'US-AL', 'US-AK', 'US-AZ', 'US-AR', 'US-CA', 'US-CO', 'US-CT', 'US-DE', 'US-DC', 'US-FL', 'US-GA', \
         'US-HI', 'US-ID', 'US-IL', 'US-IN', 'US-IA', 'US-KS', 'US-KY', 'US-LA', 'US-ME', 'US-MD', 'US-MA', 'US-MI', \
         'US-MN', 'US-MS', 'US-MO', 'US-MT', 'US-NE', 'US-NV', 'US-NH', 'US-NJ', 'US-NM', 'US-NY', 'US-NC', 'US-ND', \
         'US-OH', 'US-OK', 'US-OR', 'US-PA', 'US-RI', 'US-SC', 'US-SD', 'US-TN', 'US-TX', 'US-UT', 'US-VT', 'US-VA', \
         'US-WA', 'US-WV', 'US-WI', 'US-WY']


In [44]:
def get_candidates(yr):
    if yr == '2012':
        return 'obama', 'romney', 'BO', 'MR'
    elif yr == '2016':
        return 'hillary', 'trump', 'HC', 'DT'
    else:
        return 'biden', 'trump', 'JB', 'DT'

In [81]:
def agg_all1(states, yr):
    topics = ['economy', 'environment', 'health', 'immigration']
    
    result = []
    max_result = []
    
    for tp in topics:
        
        cand1, cand2, cs1, cs2 = get_candidates(yr)

        target_path_c1 = join('cand-state-agg', tp, yr, cand1)
        target_path_c2 = join('cand-state-agg', tp, yr, cand2)

        df1 = pd.read_csv(join(target_path_c1, 'trends.csv'))
        df2 = pd.read_csv(join(target_path_c2, 'trends.csv'))
        
        result_df = pd.merge(df1, df2, how='inner', on=['state', 'state'], \
                             suffixes=('_' + cand1, '_' + cand2))
        
        result_df = result_df.drop(['Unnamed: 0_' + cand1, 'Unnamed: 0_' + cand2], axis=1)
        
        result_df[tp] = np.where(result_df['score_' + cand1] > result_df['score_' + cand2], cs1, cs2)
        #result_df[tp + '_score'] = np.where(result_df['score_' + cand1] > result_df['score_' + cand2], \
        #                                    result_df['score_' + cand1], result_df['score_' + cand2])
        
        #result_df = result_df.drop(['score_' + cand1, 'score_' + cand2], axis=1)
        
        if len(result) == 0:
            result = result_df
        else:
            result = pd.merge(result, result_df, how='inner', on=['state', 'state'],\
                                 suffixes=('_' + tp + '1', '_' + tp + '2'))
    
    
    result['state_fips'] = result.index - 1
    result['year'] = yr
    
    print(result)
    
    result_data = result[['year', 'state_fips', 'state', 'economy', 'environment', 'health', 'immigration']]
    
    #print(result_data)
    
    return result_data

In [106]:
def agg_all(states, yr):
    topics = ['economy', 'environment', 'health', 'immigration']
    
    result = []
    result_data = []
    
    econ_result = []
    env_result = []
    health_result = []
    imm_result = []
    
    for tp in topics:
        
        cand1, cand2, cs1, cs2 = get_candidates(yr)

        target_path_c1 = join('cand-state-agg', tp, yr, cand1)
        target_path_c2 = join('cand-state-agg', tp, yr, cand2)

        df1 = pd.read_csv(join(target_path_c1, 'trends.csv'))
        df2 = pd.read_csv(join(target_path_c2, 'trends.csv'))
        
        for index, row in df1.iterrows():
            cand1_score = row['score']
            cand2_score = float(df2.iloc[index].score)
             
            if tp == 'economy':
                econ_result.append({'state_fips': index, 'year': yr, \
                                    tp: cs1 if cand1_score > cand2_score else cs2 })
            elif tp == 'environment':
                env_result.append({'state_fips': index, 'year': yr, \
                                    tp: cs1 if cand1_score > cand2_score else cs2 })
            elif tp == 'health':
                health_result.append({'state_fips': index, 'year': yr, \
                                    tp: cs1 if cand1_score > cand2_score else cs2 })
            elif tp == 'immigration':
                imm_result.append({'state_fips': index, 'year': yr, \
                                    tp: cs1 if cand1_score > cand2_score else cs2 })
            
        
            
    print(econ_result)
    print(env_result)
    print(health_result)
    print(imm_result)
            
        #result_df = pd.merge(df1, df2, how='inner', on=['state', 'state'], \
        #                     suffixes=('_' + cand1, '_' + cand2))
        
        #result_df = result_df.drop(['Unnamed: 0_' + cand1, 'Unnamed: 0_' + cand2], axis=1)
        
        #result_df[tp] = np.where(result_df['score_' + cand1] > result_df['score_' + cand2], cs1, cs2)
        #result_df[tp + '_score'] = np.where(result_df['score_' + cand1] > result_df['score_' + cand2], \
        #                                    result_df['score_' + cand1], result_df['score_' + cand2])
        
        #result_df = result_df.drop(['score_' + cand1, 'score_' + cand2], axis=1)
        
        #if len(result) == 0:
        #    result = result_df
        #else:
        #    result = pd.merge(result, result_df, how='inner', on=['state', 'state'],\
        #                         suffixes=('_' + tp + '1', '_' + tp + '2'))
    
    
    #result['state_fips'] = result.index - 1
    #result['year'] = yr
    
    #print(result)
    
    #result_data = result[['year', 'state_fips', 'state', 'economy', 'environment', 'health', 'immigration']]
    
    #print(result_data)
    
    return result_data

In [107]:
result_20 = agg_all(states, '2020')

[{'state_fips': 0, 'year': '2020', 'economy': 'JB'}, {'state_fips': 1, 'year': '2020', 'economy': 'JB'}, {'state_fips': 2, 'year': '2020', 'economy': 'JB'}, {'state_fips': 3, 'year': '2020', 'economy': 'DT'}, {'state_fips': 4, 'year': '2020', 'economy': 'JB'}, {'state_fips': 5, 'year': '2020', 'economy': 'JB'}, {'state_fips': 6, 'year': '2020', 'economy': 'DT'}, {'state_fips': 7, 'year': '2020', 'economy': 'DT'}, {'state_fips': 8, 'year': '2020', 'economy': 'JB'}, {'state_fips': 9, 'year': '2020', 'economy': 'JB'}, {'state_fips': 10, 'year': '2020', 'economy': 'DT'}, {'state_fips': 11, 'year': '2020', 'economy': 'DT'}, {'state_fips': 12, 'year': '2020', 'economy': 'JB'}, {'state_fips': 13, 'year': '2020', 'economy': 'JB'}, {'state_fips': 14, 'year': '2020', 'economy': 'DT'}, {'state_fips': 15, 'year': '2020', 'economy': 'DT'}, {'state_fips': 16, 'year': '2020', 'economy': 'JB'}, {'state_fips': 17, 'year': '2020', 'economy': 'JB'}, {'state_fips': 18, 'year': '2020', 'economy': 'JB'}, {'

In [76]:
result_16 = agg_all(states, '2016')

    year  state_fips  state economy environment health immigration
0   2016          -1     US      HC          HC     HC          HC
1   2016           0  US-AL      DT          DT     DT          DT
2   2016           1  US-AK      HC          HC     HC          HC
3   2016           2  US-AZ      DT          DT     DT          DT
4   2016           3  US-AR      DT          DT     DT          DT
5   2016           4  US-CA      HC          HC     HC          HC
6   2016           5  US-CO      HC          HC     HC          DT
7   2016           6  US-CT      DT          DT     DT          DT
8   2016           7  US-DE      HC          HC     HC          HC
9   2016           8  US-DC      HC          HC     HC          HC
10  2016           9  US-FL      HC          HC     HC          HC
11  2016          10  US-GA      HC          HC     HC          HC
12  2016          11  US-HI      HC          HC     HC          HC
13  2016          12  US-ID      HC          HC     HC        

In [77]:
result_12 = agg_all(states, '2012')

    year  state_fips  state economy environment health immigration
0   2012          -1     US      BO          BO     BO          BO
1   2012           0  US-AL      BO          BO     BO          BO
2   2012           1  US-AK      MR          MR     MR          MR
3   2012           2  US-AZ      BO          BO     BO          BO
4   2012           3  US-AR      BO          BO     BO          BO
5   2012           4  US-CA      BO          BO     BO          BO
6   2012           5  US-CO      BO          BO     BO          BO
7   2012           6  US-CT      BO          BO     BO          BO
8   2012           7  US-DE      MR          MR     MR          MR
9   2012           8  US-DC      BO          BO     BO          BO
10  2012           9  US-FL      MR          MR     BO          BO
11  2012          10  US-GA      BO          BO     BO          BO
12  2012          11  US-HI      BO          BO     BO          BO
13  2012          12  US-ID      MR          MR     MR        

In [50]:
result_combined = pd.concat([result_20, result_16, result_12])

In [51]:
result_combined

,year,state_fips,economy,environment,health,immigration
0,2020,-1,JB,JB,JB,DT
1,2020,0,JB,JB,JB,DT
2,2020,1,JB,JB,JB,DT
3,2020,2,DT,DT,DT,DT
4,2020,3,JB,JB,JB,DT
...,...,...,...,...,...,...
47,2012,46,BO,BO,BO,BO
48,2012,47,BO,BO,BO,BO
49,2012,48,MR,MR,MR,MR
50,2012,49,BO,BO,BO,BO


In [52]:
result_combined.to_csv('cand-state-agg/top_candidates.csv')